# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298688639522                   -0.85    4.0
  2   -8.300229821901       -2.81       -1.25    1.5
  3   -8.300449558222       -3.66       -1.87    4.5
  4   -8.300462174217       -4.90       -2.71    1.5
  5   -8.300464060609       -5.72       -3.11    3.6
  6   -8.300464327999       -6.57       -3.25    9.1
  7   -8.300464467943       -6.85       -3.37    1.9
  8   -8.300464556978       -7.05       -3.50    1.0
  9   -8.300464632728       -7.12       -3.78    1.2
 10   -8.300464638672       -8.23       -3.91    1.9
 11   -8.300464642954       -8.37       -4.25    1.2


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67495630073                   -0.70    6.5
  2   -16.67874004451       -2.42       -1.14    1.0
  3   -16.67922954328       -3.31       -1.86    3.6
  4   -16.67928158072       -4.28       -2.77    3.2
  5   -16.67928612174       -5.34       -3.22    4.1
  6   -16.67928621695       -7.02       -3.51    3.5
  7   -16.67928622291       -8.22       -3.98    1.0
  8   -16.67928622431       -8.86       -4.63    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31657212153                   -0.56    6.2
  2   -33.32371360858       -2.15       -1.00    1.0
  3   -33.33589284449       -1.91       -1.63    5.2
  4   -33.33686987661       -3.01       -2.32    4.1
  5   -33.33693673329       -4.17       -2.79   10.6
  6   -33.33694377852       -5.15       -3.61    2.4
  7   -33.33694391078       -6.88       -4.00    9.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298575483201                   -0.85    4.1
  2   -8.300288570453       -2.77       -1.59    1.0
  3   -8.300452338239       -3.79       -2.57    7.1
  4   -8.300384817110   +   -4.17       -2.31   12.8
  5   -8.300464496252       -4.10       -3.69    2.6
  6   -8.300464588096       -7.04       -3.93    5.6
  7   -8.300464643334       -7.26       -4.90    3.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32582645271                   -0.56    6.5
  2   -33.32496060034   +   -3.06       -1.27    1.2
  3   -21.22515969781   +    1.08       -0.53    8.5
  4   -33.28215999397        1.08       -1.62    7.8
  5   -32.99289029870   +   -0.54       -1.18    5.6
  6   -30.99200420995   +    0.30       -0.88    5.8
  7   -33.33573870833        0.37       -2.21    6.9
  8   -33.33182306687   +   -2.41       -2.08    3.1
  9   -33.33595649371       -2.38       -2.47    2.5
 10   -33.33608108069       -3.90       -2.54    1.5
 11   -33.33664804732       -3.25       -2.76    1.9
 12   -33.33693742264       -3.54       -3.42    3.5
 13   -33.33694227294       -5.31       -3.62    3.8
 14   -33.33694374577       -5.83       -3.82    2.6
 15   -33.33694365704   +   -7.05       -4.20    3.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.